# Nuclei Segmentation for [LMRG Image Analysis Study](https://sites.google.com/view/lmrg-image-analysis-study)

## Laura Cooper, University of Warwick, 24th March 2020

Based on the ["In a Notebook"](https://cellpose.readthedocs.io/en/latest/notebook.html) example from Cellpose: 

### Set up

In [1]:
# Gui for 3D rendering
%gui qt5
# Packages
import numpy as np
import time, os, sys
from cellpose import utils, models, io
from PIL import Image
from PIL.TiffTags import TAGS
from skimage import measure
import pandas as pd
import napari

# Data available from https://drive.google.com/drive/folders/1Wu-AU2-l8yEufXliD499WIiDP_0jcKlw
loc='nuclei/'
files = ['nuclei1_out_c00_dr90_image.tif',
         'nuclei2_out_c90_dr90_image.tif',
         'nuclei3_out_c00_dr10_image.tif',
         'nuclei4_out_c90_dr10_image.tif']

2021-11-16 13:22:07,153 [INFO] WRITING LOG OUTPUT TO /home/laura/.cellpose/run.log


### View Example Image

In [2]:
#change to choose file to render
img = io.imread(loc+files[0])
viewer = napari.view_image(img,colormap='gray',name='image',rendering='iso')

### Define Functions

In [3]:
def readMetadata(imageFile):
    '''
    Read metadata from imageFile
    Inputs: 
            imageFile    String, image file to read Metadata from
    Outputs:
            res          Tuple, resolution in (x, y, z)
            unit         String, measurement units of image
    '''
    with Image.open(loc+filename) as img:
        meta_dict = {TAGS[key] : img.tag[key] for key in img.tag.keys()}
        x_res=1/(meta_dict['XResolution'][0][0]/meta_dict['XResolution'][0][1])
        y_res=1/(meta_dict['YResolution'][0][0]/meta_dict['YResolution'][0][1])
        z_res=float(meta_dict['ImageDescription'][0].split()[4].split('=')[1])
        res=(x_res,y_res,z_res)
        unit=meta_dict['ImageDescription'][0].split()[3].split('=')[1]
    return res, unit

In [4]:
def runCellpose(imageFile, res):
    '''
    Open file, run cellpose and save results
    Inputs: 
            imageFile    String, image file to read Metadata from
            res          Tuple, resolution in (x, y, z)
    Outputs:
            masks        Numpy array, labelled image of segmented nuclei
            img          Numpy array, original image
    '''
    #Load nuclei model from Cellpose and set channel to be segmented
    model = models.Cellpose(gpu=False, model_type='nuclei')
    channels = [[0,0]]
    img = io.imread(loc+filename)
    masks, flows, styles, diams = model.eval(img, diameter=75, channels=channels, do_3D=True, anisotropy=res[2]/res[0])
    # save results
    io.masks_flows_to_seg(img, masks, flows, diams, filename, channels)
    return masks, img

In [5]:
def measureProperties(masks, img, res, unit):
    '''
    Measure properties of segmented nuclei and save to file
    Inputs:
            masks        Numpy array, labelled image of segmented nuclei
            img          Numpy array, original image
            res          Tuple, resolution in (x, y, z)
            unit         String, measurement units of image
    Outputs:
            df           Pandas DataFrame, table of measurements
    '''
    #Calculate properties
    properties = ['centroid', 'area']
    tables = measure.regionprops_table(masks, properties=properties)
    table1 = pd.DataFrame(tables)
    table1.columns=['z','x','y','volume']
    properties = ['intensity_image']
    props = measure.regionprops_table(masks, img, properties=properties)
    table2=pd.DataFrame([np.sum(seg) for seg in props['intensity_image']])
    table2.columns=['intensity']
    #Rescale results to micrometers and organise
    df=pd.DataFrame()
    df['x ('+unit+')'] = res[0] * table1['x']
    df['y ('+unit+')'] = res[1] * table1['y']
    df['z ('+unit+')'] = res[2] * table1['z']
    df['intensity'] = table2
    df['volume ('+unit+'^3)'] = res[0]*res[1]*res[2] * table1['volume']
    #Save CSV file
    df.to_csv('Cooper_Laura_'+filename.split('_')[0]+'.csv', index=False)
    return df

### Analysis

In [6]:
# Run cellpose, looping through files
results = dict()
for filename in files:
    res, unit = readMetadata(loc+filename)
    masks, img = runCellpose(loc+filename, res)
    results[filename]=measureProperties(masks, img, res, unit)

2021-11-16 13:22:24,377 [INFO] >>>> using CPU
2021-11-16 13:22:24,505 [INFO] ~~~ FINDING MASKS ~~~
2021-11-16 13:22:24,506 [INFO] multi-stack tiff read in as having 100 planes 1 channels
2021-11-16 13:22:24,858 [INFO] running YX: 100 planes of size (258, 258)
2021-11-16 13:22:48,277 [INFO] 100%|##########| 4/4 [00:05<00:00,  1.46s/it]
2021-11-16 13:22:48,402 [INFO] running ZY: 258 planes of size (100, 258)
2021-11-16 13:23:29,732 [INFO] 100%|##########| 9/9 [00:10<00:00,  1.14s/it]
2021-11-16 13:23:30,120 [INFO] running ZX: 258 planes of size (100, 258)
2021-11-16 13:24:03,591 [INFO] 22%|##2       | 2/9 [00:02<00:08,  1.28s/it]


2021-11-16 13:24:11,313 [INFO] 100%|##########| 9/9 [00:10<00:00,  1.15s/it]
2021-11-16 13:24:11,597 [INFO] network run in 106.91s
2021-11-16 13:24:17,118 [INFO] masks created in 5.52s
2021-11-16 13:24:17,628 [INFO] >>>> TOTAL TIME 113.12 sec
2021-11-16 13:24:18,859 [INFO] >>>> using CPU
2021-11-16 13:24:18,934 [INFO] ~~~ FINDING MASKS ~~~
2021-11-16 13:24:18,935 [INFO] multi-stack tiff read in as having 100 planes 1 channels
2021-11-16 13:24:19,272 [INFO] running YX: 100 planes of size (258, 258)
2021-11-16 13:24:43,451 [INFO] 100%|##########| 4/4 [00:06<00:00,  1.55s/it]
2021-11-16 13:24:43,563 [INFO] running ZY: 258 planes of size (100, 258)
2021-11-16 13:25:26,356 [INFO] 100%|##########| 9/9 [00:10<00:00,  1.19s/it]
2021-11-16 13:25:26,651 [INFO] running ZX: 258 planes of size (100, 258)
2021-11-16 13:25:52,295 [INFO] 33%|###3      | 3/9 [00:04<00:08,  1.38s/it]


2021-11-16 13:26:09,505 [INFO] 100%|##########| 9/9 [00:09<00:00,  1.11s/it]
2021-11-16 13:26:09,734 [INFO] network run in 110.66s
2021-11-16 13:26:15,273 [INFO] masks created in 5.54s
2021-11-16 13:26:15,884 [INFO] >>>> TOTAL TIME 116.95 sec
2021-11-16 13:26:17,532 [INFO] >>>> using CPU
2021-11-16 13:26:17,611 [INFO] ~~~ FINDING MASKS ~~~
2021-11-16 13:26:17,612 [INFO] multi-stack tiff read in as having 100 planes 1 channels
2021-11-16 13:26:17,952 [INFO] running YX: 100 planes of size (258, 258)
2021-11-16 13:26:41,585 [INFO] 100%|##########| 4/4 [00:05<00:00,  1.30s/it]
2021-11-16 13:26:41,681 [INFO] running ZY: 258 planes of size (100, 258)
2021-11-16 13:27:23,132 [INFO] 100%|##########| 9/9 [00:09<00:00,  1.08s/it]
2021-11-16 13:27:23,417 [INFO] running ZX: 258 planes of size (100, 258)
2021-11-16 13:27:38,135 [INFO] 44%|####4     | 4/9 [00:04<00:05,  1.07s/it]


2021-11-16 13:28:05,064 [INFO] 100%|##########| 9/9 [00:10<00:00,  1.12s/it]
2021-11-16 13:28:05,354 [INFO] network run in 107.60s
2021-11-16 13:28:11,040 [INFO] masks created in 5.68s
2021-11-16 13:28:11,550 [INFO] >>>> TOTAL TIME 113.94 sec
2021-11-16 13:28:12,860 [INFO] >>>> using CPU
2021-11-16 13:28:12,939 [INFO] ~~~ FINDING MASKS ~~~
2021-11-16 13:28:12,940 [INFO] multi-stack tiff read in as having 100 planes 1 channels
2021-11-16 13:28:13,280 [INFO] running YX: 100 planes of size (258, 258)
2021-11-16 13:28:37,832 [INFO] 100%|##########| 4/4 [00:05<00:00,  1.41s/it]
2021-11-16 13:28:37,932 [INFO] running ZY: 258 planes of size (100, 258)
2021-11-16 13:29:19,540 [INFO] 100%|##########| 9/9 [00:09<00:00,  1.09s/it]
2021-11-16 13:29:19,800 [INFO] running ZX: 258 planes of size (100, 258)
2021-11-16 13:29:26,034 [INFO] 56%|#####5    | 5/9 [00:06<00:04,  1.17s/it]


2021-11-16 13:30:02,502 [INFO] 100%|##########| 9/9 [00:10<00:00,  1.18s/it]
2021-11-16 13:30:02,729 [INFO] network run in 109.64s
2021-11-16 13:30:07,859 [INFO] masks created in 5.13s
2021-11-16 13:30:08,378 [INFO] >>>> TOTAL TIME 115.44 sec


### Results

In [7]:
results[files[0]]

,x (micron),y (micron),z (micron),intensity,volume (micron^3)
0,8.953426,22.792475,4.199301,66364404,578.629331
1,9.017366,5.716022,5.415583,75119277,643.622817
2,23.530731,13.620419,8.764545,89603018,656.893738
3,25.876765,22.609172,9.675107,80082276,599.999530
4,8.221444,24.882925,13.176958,68017040,599.171822


In [8]:
#change to choose file to render
dat = np.load('nuclei1_out_c00_dr90_image_seg.npy', allow_pickle=True).item()
viewer = napari.view_image(dat['img'],colormap='gray',name='image',rendering='iso')
viewer.add_image(dat['masks'],colormap='turbo',name='mask',opacity=0.5)

<Image layer 'mask' at 0x7f72ca386fd0>

2021-11-16 13:32:19,164 [INFO] No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'
